In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/db_assignment"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/db_assignment


In [3]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.precision", 2)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandarallel import pandarallel
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from contractions import fix as contractions_fix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Ensure you have downloaded these nltk resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df = pd.read_csv("./data/ubuntu_customer_msg_small.csv")

In [6]:
df.head()

,Unnamed: 0,raw_text
0,0,"Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?"
1,1,is there any reason why my 'network manager' icon in my tray does not scale when the other ones get resized when i change the size of the panel?
2,2,"I installed the 64bit version of ubuntu and I can't open firefox(segfault) and if I try to open nautilus nothing happens and my cpu goes 100%, what can I do???"
3,3,Hello Does Ubuntu have somekind of register to configure applications and os settings?
4,4,anyone else run into issues with cd/dvd burners not identifying blank media installed?


In [7]:
df = df.drop(columns=["Unnamed: 0"], axis=1)
print(df.shape)

(185762, 1)


In [8]:
print(df.isnull().sum())

raw_text    0
dtype: int64


In [9]:
sample = df.sample(10).reset_index(drop=True)

Basic Text cleaning function

In [10]:
def preprocess_text(text: str) -> str:
    """Preprocess the text"""
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Handle contractions
    text = contractions_fix(text)

    # Remove links
    text = re.sub(r"http\S+", "http-link", text)

    # Remove special characters and numbers
    text = re.sub(r"[^A-Za-z0-9\s/?/./']", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Tokenize
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [
        word
        for word in tokens
        if word not in stop_words
        or word
        in [
            "what",
            "how",
            "why",
            "where",
            "do",
            "does",
            "did",
            "is",
            "does",
            "do",
            "can",
            "could",
            "would",
            "should",
            "not",
        ]
    ]

    # Reconstruct the text
    text = " ".join(tokens)

    return text

In [11]:
# sample["clean_text"] = sample["raw_text"].parallel_apply(preprocess_text)

In [12]:
# sample

In [13]:
df["clean_text"] = df["raw_text"].parallel_apply(preprocess_text)

<ipython-input-10-6c4d41af8d1b>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-10-6c4d41af8d1b>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-10-6c4d41af8d1b>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-10-6c4d41af8d1b>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-inp

In [14]:
print(df.shape)
print(df.isnull().sum())

(185762, 2)
raw_text      0
clean_text    0
dtype: int64


In [15]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from concurrent.futures import ThreadPoolExecutor, as_completed

In [16]:
bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [17]:
# dummy_sentences = sample["clean_text"].tolist()
# print(len(dummy_sentences))

Tokenizer

In [18]:
def data_encoder(sentence: str, bert_tokenizer: BertTokenizer):
    """Encode the input sentence using the BERT tokenizer"""

    indexed_tokens = bert_tokenizer.encode(
        sentence,
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        return_tensors="pt",
    )

    segments = torch.ones_like(indexed_tokens)

    return indexed_tokens, segments

Model Prediction

In [19]:
def model_output(
    bert_model: BertModel, indexed_tokens: torch.Tensor, segments: torch.Tensor
):
    """Get the model output for the input tokens"""

    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(indexed_tokens, token_type_ids=segments)
        hidden_states = outputs[2]

    token_embeddings = torch.stack(
        hidden_states, dim=0
    )  # (layer, batch, token, feature)

    return token_embeddings

Word  and sentence token

In [20]:
def get_sentence_embedding(token_embeddings: torch.Tensor):
    """Get the sentence embedding from the token embeddings"""
    token_embeddings = token_embeddings.squeeze(1)  #  (layer, token, feature)
    token_embeddings = token_embeddings.permute(1, 0, 2)  # (token, layer, feature)
    # Stores the token vectors, with shape [seq_len x features]
    token_vecs_sum = []

    # `token_embeddings` is a [seq_len, layer, feature] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)

    print("Shape is: %d x %d" % (len(token_vecs_sum), len(token_vecs_sum[0])))
    sentence_embedding = torch.stack(token_vecs_sum).mean(dim=0).flatten()
    print("Final shape of sentence is: ", sentence_embedding.shape)
    return sentence_embedding

In [21]:
def get_sentence_embedding_opt(token_embeddings: torch.Tensor):
    """Get the sentence embedding from the token embeddings"""
    # Sum the last four layers
    sum_last_four_layers = torch.sum(
        token_embeddings[-4:], dim=0
    )  # (batch_size, seq_len, feature_dim)
    print("Shape of sum_last_four_layers: ", sum_last_four_layers.shape)
    # Mean of all tokens in the sentence
    sentence_embedding = torch.mean(
        sum_last_four_layers, dim=1
    )  # (batch_size, feature_dim)
    sentence_embedding = sentence_embedding.squeeze(0)
    print("Shape of sentence_embedding: ", sentence_embedding.shape)
    return sentence_embedding

In [22]:
# import torch

# # Create a sample input tensor (for testing)
# N_layers = 12
# batch_size = 1
# seq_len = 10
# feature_dim = 768
# sample_input = torch.randn(N_layers, batch_size, seq_len, feature_dim)

# # Test the original function
# output1 = get_sentence_embedding(sample_input.clone())

# # Test the optimized function
# output2 = get_sentence_embedding_opt(sample_input.clone())

# # Compare the results
# print(torch.allclose(output1, output2, atol=1e-6))

for each token of our input we have 13 separate vectors each of length 768 
In order to get the individual vectors we will need to combine some of the layer vectors…but which layer or combination of layers provides the best representation
let’s try creating the word vectors by summing together the last four layers.

For the entire sentence what we can do is to take the mean of the last 4 layers for each token and then take the mean of all the tokens to get a single vector representation of the sentence.

In [23]:
# demo_txt = sample["clean_text"].iloc[5]
# print(demo_txt)

In [24]:
# indexed_tokens, segments = data_encoder(demo_txt, bert_tokenizer)
# token_embeddings = model_output(bert_model, indexed_tokens, segments)
# sentence_embedding = get_sentence_embedding(token_embeddings)
# sentence_embedding_2 = get_sentence_embedding_opt(token_embeddings)

In [25]:
def get_bert_embedding(
    sentence: str, bert_tokenizer: BertTokenizer, bert_model: BertModel
):
    """Get the BERT embedding for the input sentence"""

    indexed_tokens, segments = data_encoder(sentence, bert_tokenizer)
    token_embeddings = model_output(bert_model, indexed_tokens, segments)
    sentence_embedding = get_sentence_embedding_opt(token_embeddings)

    return sentence_embedding

In [26]:
def process_sentence(
    sentence: str, bert_tokenizer: BertTokenizer, bert_model: BertModel
):
    """Process a single sentence and return the sentence and its embedding"""
    embedding = get_bert_embedding(sentence, bert_tokenizer, bert_model)
    return sentence, embedding


In [27]:


# def process_sentences(
#     sentences: list, bert_tokenizer: BertTokenizer, bert_model: BertModel
# ):
#     """Process a list of sentences in parallel and store results in a dictionary"""
#     results = {}
#     with ThreadPoolExecutor() as executor:
#         future_to_sentence = {
#             executor.submit(
#                 process_sentence, sentence, bert_tokenizer, bert_model
#             ): sentence
#             for sentence in sentences
#         }
#         for future in as_completed(future_to_sentence):
#             sentence = future_to_sentence[future]
#             try:
#                 _, embedding = future.result()
#                 results[sentence] = embedding
#             except Exception as exc:
#                 print(f"{sentence} generated an exception: {exc}")
#     return results

In [28]:
from joblib import Parallel, delayed
from functools import partial

In [29]:
def process_sentences(
    sentences: list, bert_tokenizer: BertTokenizer, bert_model: BertModel, n_jobs=-1
):
    """Process a list of sentences in parallel and store results in a dictionary"""
    # Create a partial function with bert_tokenizer and bert_model
    process_with_model = partial(
        process_sentence, bert_tokenizer=bert_tokenizer, bert_model=bert_model
    )
    results = Parallel(n_jobs=n_jobs, timeout=100)(
        delayed(process_with_model)(sentence) for sentence in sentences
    )
    return dict(results)

In [30]:
all_intents = df["clean_text"].tolist()
print(len(all_intents))

185762


In [31]:
all_intents_embeddings = process_sentences(all_intents, bert_tokenizer, bert_model)

/anaconda/envs/torch_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/anaconda/envs/torch_env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
# # apply all these functions to the column cleantext of the df
# df["bert_embedding"] = df["clean_text"].apply(
#     lambda x: get_bert_embedding(x, bert_tokenizer, bert_model)
# )

In [ ]:
os.makedirs("./data/pre_processed", exist_ok=True)
df.to_csv(
    "./data/pre_processed/ubuntu_customer_msg_small_pre_processed_bert_base.csv",
    index=False,
)

In [ ]:
X_cls_bert = np.vstack(sample['bert_embedding'])

In [ ]:
X_cls_bert.shape

In [ ]:
def cluster_embeddings(embeddings, num_clusters):
    scaler = StandardScaler()
    normalized_embeddings = scaler.fit_transform(embeddings)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    clusters = kmeans.fit_predict(normalized_embeddings)
    return clusters, kmeans

In [ ]:
def evaluate_clustering(embeddings, clusters):
    silhouette_avg = silhouette_score(embeddings, clusters)
    return silhouette_avg